<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/KFood_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv
from glob import glob

In [9]:
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32

In [10]:
#데이터셋
DATASET_NAME = 'kfood'
DRIVE_PATH = '/content/drive/MyDrive/Datasets'

DATASET_PATH = DRIVE_PATH + "/" + DATASET_NAME
filepath = DATASET_PATH
print(os.path.exists(filepath))

True


In [64]:
#class_label 매칭 딕셔너리로 저장
labels = None
classes = None
with open(os.path.join(filepath, 'class_label.csv'),'r') as f:
    w = csv.reader(f)
    classes = w.__next__()
    labels = w.__next__()
#print(len(classes), len(labels), classes, labels)
label_to_class = [0]*len(labels)
for _class, _label in zip(classes, labels):
    label_to_class[int(_label)] = _class.encode('utf-8')

class_to_label = {_class: _label for _label, _class in enumerate(label_to_class)}

tf_class_to_label = tf.constant(list(class_to_label.keys()))


In [69]:
class_paths = sorted(glob(os.path.join(filepath, '*', '*')))
class_names = [class_path.split('/')[-1] for class_path in class_paths]


KeyError: ignored

In [72]:
label_paths = sorted(glob(os.path.join(filepath, '*', '*')))
label_paths = [label_path.split('/')[-1] for label_path in label_paths if os.path.isdir(label_path)]
label_paths[0]

'갈비구이'

In [74]:
with open(os.path.join(filepath, 'class_label.csv'),'w', encoding='utf-8') as f:
    w = csv.writer(f) 
    w.writerow(list(range(len(label_paths))))
    w.writerow(label_paths)

In [76]:
with open(os.path.join(filepath, 'class_label.csv'),'r') as f:
    w = csv.reader(f)
    classes = w.__next__()
    labels = w.__next__()

print(labels)
print(classes)

['갈비구이', '갈치구이', '고등어구이', '곱창구이', '닭갈비', '더덕구이', '떡갈비', '불고기', '삼겹살', '장어구이', '조개구이', '조기구이', '황태구이', '훈제오리', '계란국', '떡국_만두국', '무국', '미역국', '북엇국', '시래기국', '육개장', '콩나물국', '과메기', '양념치킨', '젓갈', '콩자반', '편육', '피자', '후라이드치킨', '갓김치', '깍두기', '나박김치', '무생채', '배추김치', '백김치', '부추김치', '열무김치', '오이소박이', '총각김치', '파김치', '가지볶음', '고사리나물', '미역줄기볶음', '숙주나물', '시금치나물', '애호박볶음', '경단', '꿀떡', '송편', '만두', '라면', '막국수', '물냉면', '비빔냉면', '수제비', '열무국수', '잔치국수', '짜장면', '짬뽕', '쫄면', '칼국수', '콩국수', '꽈리고추무침', '도라지무침', '도토리묵', '잡채', '콩나물무침', '홍어무침', '회무침', '김밥', '김치볶음밥', '누룽지', '비빔밥', '새우볶음밥', '알밥', '유부초밥', '잡곡밥', '주먹밥', 'ᄀ

In [ ]:
def get_image_crop_points(filepath):
    crops = {}
    properties = os.path.join(filepath, "crop_area.properties")
    with open(properties, 'r') as p:
        for row in p:
            name, crop = row.replace("\n", "").replace(" ", "").split("=")
            if name != "" and crop != "":
                #name = name.encode('utf-8')
                crop = crop.split(",")
                if len(crop) >= 4:
                    crop = [int(crop[1]), int(crop[0]), int(crop[3]), int(crop[2])]
                    crops[name] = crop
                elif len(crop) == 2:
                    crop = [0, 0, int(crop[1]), int(crop[0])]
                    crops[name] = crop
            
    return crops

In [ ]:
#crop 지점 정보 빼오기
crop_points = {}
class_list = list(glob( os.path.join(filepath, "*", "*")))
class_list = [class_name for class_name in class_list if os.path.isdir(class_name)]
for class_name in class_list:
    crop_points.update(get_image_crop_points(class_name))

tf_crop_image_names = tf.constant(list(crop_points.keys()), dtype=tf.string)
tf_crop_points = tf.constant(list(crop_points.values()))


In [ ]:
#데이터셋의 이미지 경로 및 레이블 저장
image_paths = sorted(glob(os.path.join(filepath, "*", "*", "*")))
#image_paths = sorted(list(DATASET_PATH.glob('*/*/*')))
image_paths = [image_path for image_path in image_paths if image_path.split('/')[-1].split('.')[-1].lower() in ("png", "jpg", "jpeg")]
#labels = [class_to_label[Path(image_path).parent.stem] for image_path in image_paths]
print(len(image_paths))#, len(labels))

150649


In [ ]:
#filepath = tf.compat.path_to_str(image_paths[0])
filepath = image_paths[0]
tf.strings.lower(tf.strings.split(filepath, ".")[-1])
tf.strings.split(tf.strings.split(filepath, "/")[-1], ".")[0]
tf.strings.split(filepath, "/")[-2]

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe1\x84\x80\xe1\x85\xa1\xe1\x86\xaf\xe1\x84\x87\xe1\x85\xb5\xe1\x84\x80\xe1\x85\xae\xe1\x84\x8b\xe1\x85\xb5'>

In [ ]:
def parse_and_crop_image_add_label(tf_filepath):
    
    image = tf.io.read_file(tf_filepath) # 이미지 파일 읽기
    filepath = tf_filepath
    #format decoding
    image_format = tf.strings.lower(tf.strings.split(filepath, ".")[-1])

    if image_format == "jpeg":
        image = tf.image.decode_jpeg(image) # JPEG-encoded -> uint8 tensor (RGB format)
    elif image_format == "png":
        image = tf.image.decode_png(image, channels=3, dtype=tf.uint8)

    else:
        image = tf.image.decode_image(image, channels=3, expand_animations=False)
    


    #crop
    image_name = tf.strings.split(tf.strings.split(filepath, "/")[-1], ".")[0]
    tf_image_idx = tf.where(tf_crop_image_names == image_name)
    
    #crop 정보가 있으면 크롭
    if tf.reduce_all(tf.not_equal(tf.shape(tf_image_idx), tf.constant((0, 1), dtype=tf.int32))):
        crop_offsets = tf_crop_points[tf.reshape(tf_image_idx, shape=())]
        image = tf.image.crop_to_bounding_box(image, crop_offsets[0], crop_offsets[1], crop_offsets[2], crop_offsets[3])
    
    #labeling
    class_name = tf.strings.split(filepath, "/")[-2]
    tf_class_name_idx = tf.where(tf_class_to_label == class_name)
    
    try:
        label = tf.reshape(tf_class_name_idx, shape=())
    except:
        label = 0
        print("label error")

    return image, int(label)

In [ ]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 2
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 2
    right_crop = shape[1] - left_crop
    return image[top_crop : bottom_crop, left_crop : right_crop]


def resizing_image(image, label):
    image = central_crop(image)
    image = tf.image.resize(image, [299, 299], method="nearest")
    return image, label

In [ ]:
def make_kfood_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None, n_parse_threads=5, batch_size=32, cache=False):

    filenames_dataset = tf.data.Dataset.from_tensor_slices(filepaths)
    dataset = filenames_dataset.map(parse_and_crop_image_add_label, num_parallel_calls=n_parse_threads)
    dataset = dataset.map(resizing_image, num_parallel_calls=n_parse_threads)
    
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    if batch_size:
        dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [ ]:
#데이터셋 생성
dataset = make_kfood_dataset(image_paths, shuffle_buffer_size=10000, n_parse_threads=tf.data.AUTOTUNE)

In [ ]:
for x in dataset.take(1):
    print(x)

In [ ]:
def dataset_image_plot_4():
    for images, labels in dataset.take(1):
        plt.figure(figsize=(16,8))
        plt.axis("off")
        for i in range(4):
            plt.subplot(1, 4, i+1)
            image = tf.cast(images[i], dtype=tf.float32)
            plt.imshow(image / 255.)
            print(bytes(tf_class_to_label[int(labels[i])].numpy()).decode('utf-8'))
            
dataset_image_plot_4()

InvalidArgumentError: ignored

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv

IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32
#데이터셋
DATASET_NAME = 'kfood'
#DRIVE_PATH = Path(os.getcwd())
DATASET_PATH = DRIVE_PATH / DATASET_NAME
filepath = DATASET_PATH
print(filepath.exists())

#class_label 매칭 딕셔너리로 저장
labels = None
classes = None
with open(filepath / 'class_label.csv','r') as f:
    w = csv.reader(f)
    classes = w.__next__()
    labels = w.__next__()
#print(len(classes), len(labels), classes, labels)
class_to_label = {}
for _class, _label in zip(classes, labels):
    class_to_label[_class] = int(_label)

tf_class_to_label = list(class_to_label.keys())
tf_class_to_label = [utf_8.encode("utf-8") for utf_8 in tf_class_to_label]
tf_class_to_label = tf.constant(tf_class_to_label)


def get_image_crop_points(filepath):
    crops = {}
    properties = filepath / "crop_area.properties"
    with open(properties, 'r') as p:
        for row in p:
            name, crop = row.replace("\n", "").replace(" ", "").split("=")
            if name != "" and crop != "":
                #name = name.encode('utf-8')
                crop = crop.split(",")
                if len(crop) >= 4:
                    crop = [int(crop[1]), int(crop[0]), int(crop[3]), int(crop[2])]
                    crops[name] = crop
                elif len(crop) == 2:
                    crop = [0, 0, int(crop[1]), int(crop[0])]
                    crops[name] = crop
            
    return crops

#crop 지점 정보 빼오기
crop_points = {}
class_list = list(filepath.glob("*/*"))
class_list = [class_name for class_name in class_list if class_name.is_dir()]
for class_name in class_list:
    crop_points.update(get_image_crop_points(class_name))

tf_crop_image_names = tf.constant(list(crop_points.keys()), dtype=tf.string)
tf_crop_points = tf.constant(list(crop_points.values()))

#데이터셋의 이미지 경로 및 레이블 저장
from glob import glob
image_paths = sorted(glob(str(DATASET_PATH) + "/*/*/*"))
image_paths = [image_path for image_path in image_paths if image_path.split("/")[-1].split(".")[-1].lower() in ("png", "jpg", "jpeg")]
#labels = [class_to_label[Path(image_path).parent.stem] for image_path in image_paths]
print(len(image_paths))#, len(labels))


def parse_and_crop_image_add_label(tf_filepath):
    
    image = tf.io.read_file(tf_filepath) # 이미지 파일 읽기
    filepath = tf.compat.path_to_str(tf_filepath)
    #format decoding
    image_format = tf.strings.lower(tf.strings.split(filepath, ".")[-1])

    if image_format == "jpeg":
        image = tf.image.decode_jpeg(image, channels=3) # JPEG-encoded -> uint8 tensor (RGB format)
    elif image_format == "png":
        image = tf.image.decode_png(image, channels=3, dtype=tf.uint8)
    else:    
        image = tf.image.decode_image(image, channels=3, expand_animations=False)


    #crop
    image_name = tf.strings.split(tf.strings.split(filepath, "/")[-1], ".")[0]
    tf_image_idx = tf.where(tf_crop_image_names == image_name)
    
    #crop 정보가 있으면 크롭
    if tf.reduce_all(tf.not_equal(tf.shape(tf_image_idx), tf.constant((0, 1), dtype=tf.int32))):
        crop_offsets = tf_crop_points[tf.reshape(tf_image_idx, shape=())]
        image = tf.image.crop_to_bounding_box(image, crop_offsets[0], crop_offsets[1], crop_offsets[2], crop_offsets[3])
    
    #labeling
    class_name = tf.strings.split(filepath, "/")[-2]
    tf_class_name_idx = tf.where(tf_class_to_label == class_name)

    tf.print(tf_class_name_idx)
    
    try:
        label = tf.reshape(tf_class_name_idx, shape=())
    except:
        label = 0
        tf.print("label error")

    return image, int(label)

def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 2
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 2
    right_crop = shape[1] - left_crop
    return image[top_crop : bottom_crop, left_crop : right_crop]


def resizing_image(image, label):
    image = central_crop(image)
    image = tf.image.resize(image, [299, 299], method="nearest")
    return image, label

def make_kfood_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None, n_parse_threads=5, batch_size=32, cache=False):

    filenames_dataset = tf.data.Dataset.from_tensor_slices(filepaths)
    dataset = filenames_dataset.map(parse_and_crop_image_add_label, num_parallel_calls=n_parse_threads)
    dataset = dataset.map(resizing_image, num_parallel_calls=n_parse_threads)
    #dataset = filenames_dataset.map(spa)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    if batch_size:
        dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)


#데이터셋 생성
dataset = make_kfood_dataset(image_paths, shuffle_buffer_size=10000, n_parse_threads=tf.data.AUTOTUNE)

In [ ]:
tf_class_to_label
print(tf.strings.unicode_decode(tf.strings.split(image_paths[0], '/')[-2], input_encoding='UTF-8'))
text_ = tf.constant('갈비구이')
text_chars = tf.strings.unicode_decode(text_, input_encoding='UTF-8')
text_utf8 = tf.strings.unicode_encode(text_chars, output_encoding='UTF-8')
print(tf.strings.split(image_paths[0], '/'))
print(text_chars)
print(image_paths[0])
#print(text_utf8)
#print(bytes(tf.strings.split(image_paths[0], '/')[-2].numpy()).decode('utf-8'))

In [ ]:
print(b'\xe1\x84\x80\xe1\x85\xa1\xe1\x86\xaf\xe1\x84\x87\xe1\x85\xb5\xe1\x84\x80\xe1\x85\xae\xe1\x84\x8b\xe1\x85\xb5'.decode('utf-8').encode('utf-8'))


In [ ]:
print(image_paths[0].split('/')[-2].encode('utf-8'))
image_names = [image_path.split('/')[-2] for image_path in image_paths]

In [ ]:
print(image_names[0].encode('utf-8'))

In [ ]:
def dataset_image_plot_4():
    for images, labels in dataset.take(1):
        plt.figure(figsize=(16,8))
        plt.axis("off")
        for i in range(4):
            plt.subplot(1, 4, i+1)
            image = tf.cast(images[i], dtype=tf.float32)
            plt.imshow(image / 255.)
            print(bytes(tf_class_to_label[int(labels[i])].numpy()).decode('utf-8'))
            
dataset_image_plot_4()